# Set environment

## pip install new package

In [1]:
!pip install pythainlp
!pip install fasttext

     |████████████████████████████████| 11.5 MB 10.4 MB/s 
     |████████████████████████████████| 68 kB 6.3 MB/s 
  Using cached pybind11-2.9.1-py2.py3-none-any.whl (211 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3131980 sha256=6672378e3e41d35ff95416ad77a32ea847beccd01db10c668eaaf2d9a2cb00a9
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


## import package to environment 

In [2]:
#Import Package
import pandas as pd
import numpy as np
import json, re, unicodedata, string, typing, time ,os
from collections import Counter
import pickle
from pythainlp.tokenize import word_tokenize
import fasttext
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Google drive

In [3]:
#mount my google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/BADS9000_IS/Colab-DrQA

/content/drive/MyDrive/Colab Notebooks/BADS9000_IS/Colab-DrQA


## PC

In [ ]:
#For PC
os.chdir('C:/Users/taiti/OneDrive/MasterDegree/BADS9000_IS/ThaiDrQA')
print(f"Current working directory {os.getcwd()}")

Current working directory C:\Users\taiti\OneDrive\MasterDegree\BADS9000_IS\ThaiDrQA


## Function

In [5]:
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print("Length of data: ", len(data['data']))
    print("Data Keys: ", data['data'][0].keys())   
    return data

def load_article(file):
    try :
        with open(file, 'r', encoding='utf-8') as f:
            context = f.read()
    except :
        context = "-"
        print(f"Error Article id {id}")
    return  context

def gather_text_for_vocab(dfs:list):
    '''
    Gathers text from contexts and questions to build a vocabulary.
    '''
    text = []
    total = 0
    for df in dfs:
        unique_contexts = list(df.context.unique())
        unique_questions = list(df.question.unique())
        total += df.context.nunique() + df.question.nunique()
        text.extend(unique_contexts + unique_questions)
    
    assert len(text) == total #For debugging code
    print("Number of sentences in dataset: ", len(vocab_text))
    return text

def build_word_vocab(vocab_text):
    """Tokenizer Ref : https://github.com/PyThaiNLP/pythainlp """
    words = []
    for sent in vocab_text:
        for word in word_tokenize(sent ,engine='newmm'):
            words.append(word)
    word_counter = Counter(words)
    word_vocab = sorted(word_counter, key=word_counter.get, reverse=True)
    print(f"raw-vocab: {len(word_vocab)}")
    print(f"vocab-length: {len(word_vocab)}")
    word2idx = {word:idx for idx, word in enumerate(word_vocab)}
    print(f"word2idx-length: {len(word2idx)}")
    idx2word = {v:k for k,v in word2idx.items()}
    
    return word2idx, idx2word, word_vocab

def word_to_ids(text, word2idx):
    '''
    Converts word text to their respective ids by mapping each word
    using word2idx. Input text is tokenized using spacy tokenizer first.
    :param str text: context text to be converted
    :param dict word2idx: word to id mapping
    :returns list context_ids: list of mapped ids
    :raises assertion error: sanity check
    '''
    words_tokens = [w for w in word_tokenize(text ,engine='newmm') ]
    words_ids = [word2idx[word] for word in words_tokens]
    
    assert len(words_ids) == len(words_tokens)
    return words_ids

def save_to_pickle(save_obj, path_file):
    with open(path_file, 'wb') as file:
        pickle.dump(save_obj, file)
    print(f"save {path_file[27:-4]} to {path_file} success")

def load_pickle(path_file):
    with open(path_file, 'rb') as file:
        load_obj = pickle.load(file)
        print(f"load object from {path_file} success,that is {type(load_obj)}")
        return load_obj

# Built data for Training

## Load Data JSON to DataFrame

In [ ]:
#Open Json File
data = load_json('./data/ThaiQACorpus-DevelopmentDataset.json')

Length of data:  4000
Data Keys:  dict_keys(['question_id', 'question', 'answer', 'answer_begin_position ', 'answer_end_position', 'article_id'])


In [ ]:
#load data JSON to DataFrame
qa_lst = []
for qa in data['data']:
    qa_dict = {}
    qa_dict['article_id'] = qa['article_id']
    qa_dict['id'] = qa['question_id']
    qa_dict['context']  = load_article( f"./data/documents-nsc/{qa['article_id']}.txt")
    qa_dict['question'] = qa['question']
    qa_dict['label'] = [qa['answer_begin_position '],qa['answer_end_position']]
    qa_dict['answer'] = qa['answer']
    qa_lst.append(qa_dict)

df_qa = pd.DataFrame(qa_lst)
print(f"Shap:{df_qa.shape}")
display(df_qa.head(2))

Shap:(4000, 6)


,article_id,id,context,question,label,answer
0,115035,1,"<doc id=""115035"" url=""https://th.wikipedia.org...",สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Ben...,"[529, 538]",ฮิกกิ้นส์
1,376583,2,"<doc id=""376583"" url=""https://th.wikipedia.org...",ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,"[139, 144]",เมชตา


## Create list of Word

In [ ]:
# gather text to build vocabularies
%time vocab_text = gather_text_for_vocab([df_qa])
%time word2idx, idx2word, word_vocab = build_word_vocab(vocab_text)

Number of sentences in dataset:  6266
Wall time: 192 ms
raw-vocab: 63648
vocab-length: 63648
word2idx-length: 63648
Wall time: 35 s


In [ ]:
# numericalize context and questions
%time df_qa['context_ids']   = df_qa.context.apply(word_to_ids,   word2idx=word2idx)
%time df_qa['question_ids'] = df_qa.question.apply(word_to_ids,  word2idx=word2idx)
df_qa.head(2)

Wall time: 58.3 s
Wall time: 535 ms


In [ ]:
def test_indices(df, idx2word):
    '''
    Performs the tests mentioned above. This method also gets the start and end of the answers
    with respect to the context_ids for each example.
    :param dataframe df: SQUAD df
    :param dict idx2word: inverse mapping of token ids to words
    :returns
        list start_value_error: example idx where the start idx is not found in the start spans
                                of the text
        list end_value_error: example idx where the end idx is not found in the end spans
                              of the text
        list assert_error: examples that fail assertion errors. A majority are due to the above errors
    '''

    start_value_error = []
    end_value_error = []
    assert_error = []
    for index, row in df.iterrows():
        answer_tokens = [w for w in word_tokenize(row['answer'] ,engine='newmm')]
        
        context_tokens = word_tokenize(row['context'] ,engine='newmm')
        context_span  = [(len("".join(context_tokens[0:i])), len("".join(context_tokens[0:i+1]))) 
                         for i,w in enumerate(context_tokens)]
        starts, ends = zip(*context_span)

        answer_start, answer_end = (row['label'][0]-1,row['label'][1]-1)

        try:
            start_idx = starts.index(answer_start)
        except:
            start_value_error.append(index)
        try:
            end_idx  = ends.index(answer_end)
        except:
            end_value_error.append(index)

        try:
            assert idx2word[row['context_ids'][start_idx]] == answer_tokens[0]
            assert idx2word[row['context_ids'][end_idx]] == answer_tokens[-1]
        except:
            assert_error.append(index)


    return start_value_error, end_value_error, assert_error

def get_error_indices(df, idx2word):
    
    start_value_error, end_value_error, assert_error = test_indices(df, idx2word)
    err_idx = start_value_error + end_value_error + assert_error
    err_idx = set(err_idx)
    print(f"Number of error indices: {len(err_idx)}")
    
    return err_idx

In [ ]:
%%time 
train_err = get_error_indices(df_qa, idx2word)

df_qa.drop(train_err, inplace=True)
print(f"Shape of data frame after drop error row: {df_qa.shape}")
#Some row is error explore that how to fixed that#

Number of error indices: 149
Wall time: 13min 3s


In [ ]:
def index_answer(row, idx2word):
    '''
    Takes in a row of the dataframe or one training example and
    returns a tuple of start and end positions of answer by calculating 
    spans.
    '''
    context_tokens = word_tokenize(row['context'] ,engine='newmm')
    context_span  = [(len("".join(context_tokens[0:i])), len("".join(context_tokens[0:i+1]))) 
                     for i,w in enumerate(context_tokens)]
    starts, ends = zip(*context_span)
    
    answer_start, answer_end = (row['label'][0]-1,row['label'][1]-1)
    
    start_idx = starts.index(answer_start)
    end_idx  = ends.index(answer_end)
    
    answer_tokens = [w for w in word_tokenize(row['answer'] ,engine='newmm')]
    assert idx2word[row.context_ids[start_idx]] == answer_tokens[0]
    assert idx2word[row.context_ids[end_idx]] == answer_tokens[-1]
    
    return [start_idx, end_idx]

In [ ]:
%%time
#label_idx is position in context_ids ,which value convert to word by idx2word[]
label_idx = df_qa.apply(index_answer, axis=1, idx2word=idx2word)
df_qa['label_idx'] = label_idx
print(f"df_qa.shape:{df_qa.shape}")
display(df_qa.head(2))

train_df.shape:(3851, 9)


,article_id,id,context,question,label,answer,context_ids,question_ids,label_idx
0,115035,1,"<doc id=""115035"" url=""https://th.wikipedia.org...",สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Ben...,"[529, 538]",ฮิกกิ้นส์,"[101, 43, 0, 99, 27, 26953, 12, 0, 102, 27, 97...","[2036, 167, 82, 111, 6, 2705, 6355, 1, 114, 57...","[163, 164]"
1,376583,2,"<doc id=""376583"" url=""https://th.wikipedia.org...",ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,"[139, 144]",เมชตา,"[101, 43, 0, 99, 27, 26958, 12, 0, 102, 27, 97...","[1257, 188, 0, 62, 0, 6, 7727, 946, 82, 1, 487...","[55, 57]"


Wall time: 20.3 ms


## Save Data to plckle

In [ ]:
#Save this State
save_to_pickle(df_qa , "./drqa/1-tokenizers/result/df_qa.pkl")
save_to_pickle(word2idx , "./drqa/1-tokenizers/result/dict_word2idx.pkl")
save_to_pickle(idx2word , "./drqa/1-tokenizers/result/dict_idx2word.pkl")
save_to_pickle(word_vocab , "./drqa/1-tokenizers/result/list_word_vocab.pkl")

save df_qa to ./drqa/1-tokenizers/result/df_qa.pkl success
save dict_word2idx to ./drqa/1-tokenizers/result/dict_word2idx.pkl success
save dict_idx2word to ./drqa/1-tokenizers/result/dict_idx2word.pkl success
save list_word_vocab to ./drqa/1-tokenizers/result/list_word_vocab.pkl success


## Load Data

In [6]:
df_qa = load_pickle("./drqa/1-tokenizers/result/df_qa.pkl")
word2idx = load_pickle("./drqa/1-tokenizers/result/dict_word2idx.pkl")
idx2word = load_pickle("./drqa/1-tokenizers/result/dict_idx2word.pkl")
word_vocab = load_pickle("./drqa/1-tokenizers/result/list_word_vocab.pkl")

load object from ./drqa/1-tokenizers/result/df_qa.pkl success,that is <class 'pandas.core.frame.DataFrame'>
load object from ./drqa/1-tokenizers/result/dict_word2idx.pkl success,that is <class 'dict'>
load object from ./drqa/1-tokenizers/result/dict_idx2word.pkl success,that is <class 'dict'>
load object from ./drqa/1-tokenizers/result/list_word_vocab.pkl success,that is <class 'list'>


# Embedding 
ref : https://fasttext.cc/docs/en/crawl-vectors.html

## FASTTEXT

In [ ]:
%%time
th_embedding = fasttext.load_model("./data/cc.th.300.bin")
print(f"Embedding dimension : {th_embedding.get_dimension()}")
print(f"Number of word in embedding : {len(th_embedding.get_words())}")
print(f"Example word in embedding : {th_embedding.get_words()[0:15]}")

Embedding dimension : 300
Number of word in embedding : 2000000
Example word in embedding : ['ที่', '</s>', "'", '.', 'การ', '-', 'ใน', 'และ', ')', '(', 'เป็น', 'ของ', ',', 'ได้', ':']
CPU times: user 4.89 s, sys: 8.45 s, total: 13.3 s
Wall time: 1min 1s


In [ ]:
def create_embed_matrix():
  embed_matrix = {}
  for word in th_embedding.get_words():
    embed_matrix[word] = th_embedding.get_word_vector(word)
  return embed_matrix

In [ ]:
%%time
embed_matrix = create_embed_matrix()
print(f"Size of embed_matrix : {len(embed_matrix)}")

Size of embed_matrix : 2000000
CPU times: user 28.4 s, sys: 10.6 s, total: 39 s
Wall time: 33.9 s


In [ ]:
def create_word_embedding(embed_matrix):
    '''
    Creates a weight matrix of the words that are common in the GloVe vocab and
    the dataset's vocab. Initializes OOV words with a zero vector.
    '''
    weights_matrix = np.zeros((len(word_vocab), 300))
    words_found = 0
    not_found = []
    for i, word in enumerate(word_vocab):
        try:
            weights_matrix[i] = embed_matrix[word]
            words_found += 1
        except:
          not_found.append(word)
    return weights_matrix, words_found ,not_found

In [ ]:
%%time
weights_matrix, words_found, not_found = create_word_embedding(embed_matrix)
print(f"Total words found in glove vocab: {words_found}/{len(word_vocab)}", )

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.48 µs
Total words found in glove vocab: 38136/63648


## PythaiNLP : thai2fit_wv

In [24]:
from pythainlp.corpus import get_corpus_path
from gensim.models import KeyedVectors


In [36]:
path = get_corpus_path("thai2fit_wv")
thaiW2V = KeyedVectors.load_word2vec_format(path, binary=True)
print(f"number of word in corpus {len(thaiW2V.vocab.keys()):0,.0f}")
print(f"number of dimention of vector {thaiW2V.vector_size}")

number of word in corpus 51,358
number of dimention of vector 300


In [21]:
weights_matrix = np.zeros((len(word_vocab), 300))
words_found = 0
not_found = []
for i, word in enumerate(word_vocab):
  try:
    weights_matrix[i] = thaiW2V.get_vector(word)
    words_found += 1
  except:
    not_found.append(word)
print(f"Total words found in thainlp W2V: {words_found}/{len(word_vocab)}", )
print(f"Example word not found : {not_found[0:20]}")

Total words found in thainlp W2V: 29686/63648
Example word not found : [' ', 'ใน', '"', '="', '/', '://', 'curid', '\n', 'อร', '</', '">', 'ส์', 'ต่าง ๆ', 'The', 'ยังมี', 'เป็นที่', 'ฯ', 'พระองค์เจ้า', 'สเปน', 'บี']


In [37]:
path = get_corpus_path("ltw2v")
ltw2v = KeyedVectors.load_word2vec_format(path, binary=True, unicode_errors='ignore')
print(f"number of word in corpus {len(ltw2v.vocab.keys()):0,.0f}")
print(f"number of dimention of vector {ltw2v.vector_size}")

number of word in corpus 731,185
number of dimention of vector 400


In [44]:
weights_matrix = np.zeros((len(word_vocab), 400))
words_found = 0
not_found = []
for i, word in enumerate(word_vocab):
  try:
    weights_matrix[i] = ltw2v.get_vector(word)
    words_found += 1
  except:
    not_found.append(word)
print(f"Total words found in thainlp W2V: {words_found}/{len(word_vocab)}", )
print(f"Example word not found : {not_found[0:20]}")

Total words found in thainlp W2V: 52890/63648
Example word not found : [' ', 'curid', '\n', 'ต่าง ๆ', '2.', '1.', '3.', '4.', '5.', '6.', '7.', '8.', '9.', '10.', '  ', '11.', 'แฮร์รี่ พอตเตอร์', 'เสรี', '12.', '13.']


## Save data

In [ ]:
%%time
save_to_pickle(embed_matrix , "./drqa/1-tokenizers/result/dict_embed_matrix.pkl")
np.save('./drqa/1-tokenizers/result/drqaglove_vt.npy',weights_matrix)

save dict_embed_matrix to ./drqa/1-tokenizers/result/dict_embed_matrix.pkl success
CPU times: user 23.5 s, sys: 9.72 s, total: 33.2 s
Wall time: 44.3 s


# Summary

Problem


1. Lost 50% of word in QA because word aren't found in thai Embedding Fasttext
2. https://pythainlp.github.io/docs/2.2/api/word_vector.html
3. https://pythainlp.github.io/dev-docs/api/word_vector.html
4. https://pythainlp.github.io/pythainlp-corpus/list-corpus.html
4. we use original context that is html.

In [49]:
my_string = "Where's Waldo?"
my_string.index("Waldo")

8

In [50]:
df_qa.head()

,article_id,id,context,question,label,answer,context_ids,question_ids,label_idx
0,115035,1,"<doc id=""115035"" url=""https://th.wikipedia.org...",สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Ben...,"[529, 538]",ฮิกกิ้นส์,"[101, 43, 0, 99, 27, 26953, 12, 0, 102, 27, 97...","[2036, 167, 82, 111, 6, 2705, 6355, 1, 114, 57...","[163, 164]"
1,376583,2,"<doc id=""376583"" url=""https://th.wikipedia.org...",ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,"[139, 144]",เมชตา,"[101, 43, 0, 99, 27, 26958, 12, 0, 102, 27, 97...","[1257, 188, 0, 62, 0, 6, 7727, 946, 82, 1, 487...","[55, 57]"
2,376583,3,"<doc id=""376583"" url=""https://th.wikipedia.org...",ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,"[641, 649]",ลมสุริยะ,"[101, 43, 0, 99, 27, 26958, 12, 0, 102, 27, 97...","[1257, 188, 0, 62, 0, 6, 7727, 946, 82, 1, 487...","[174, 175]"
3,547560,4,"<doc id=""547560"" url=""https://th.wikipedia.org...",กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อของนาซีเยอ...,"[304, 311]",ปี 1933,"[101, 43, 0, 99, 27, 26960, 12, 0, 102, 27, 97...","[1023, 700, 5848, 2, 12679, 5, 2223, 429, 0, 8...","[72, 74]"
4,193032,5,"<doc id=""193032"" url=""https://th.wikipedia.org...",ห้องทำงานรูปไข่ สร้างขึ้นในสมัยอดีตประธานาธิบด...,"[231, 262]",ประธานาธิบดีวิลเลียม เอช. ทัฟต์,"[101, 43, 0, 99, 27, 26961, 12, 0, 102, 27, 97...","[9062, 5726, 0, 92, 54, 1, 214, 202, 293, 3444...","[63, 69]"


In [55]:
from bs4 import BeautifulSoup

In [69]:
idx = 4
text = BeautifulSoup(df_qa.context[idx], 'lxml').text
start_idx = text.index(df_qa.answer[idx])
print(text[start_idx:start_idx+len(df_qa.answer[idx])])

ประธานาธิบดีวิลเลียม เอช. ทัฟต์
